In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import numpy as np
import keras

import sys
sys.path.append("scripts")
import utils
from PET import PET

import matplotlib.pyplot as plt

In [ ]:
tf.config.run_functions_eagerly(False)

In [ ]:
import awkward
import vector

def to_p4(p4_obj):
    return vector.awk(
        awkward.zip(
            {
                "mass": p4_obj.tau,
                "x": p4_obj.x,
                "y": p4_obj.y,
                "z": p4_obj.z,
            }
        )
    )

def deltaphi(phi1, phi2):
    diff = phi1 - phi2
    return np.arctan2(np.sin(diff), np.cos(diff))

def deltar(eta1, phi1, eta2, phi2):
    deta = eta1 - eta2
    dphi = deltaphi(phi1, phi2)
    return np.sqrt(deta**2 + dphi**2)

Define the backbone model

In [ ]:
model = PET(
    num_feat=13,
    num_jet=4,
    num_classes=10,
    local=True,
    num_layers=8,
    drop_probability=0,
    simple=False,
    layer_scale=True,
    talking_head=False,
    mode="all"
)

Run the backbone model on dummy data to initialize weights.

In [ ]:
x = {}
x["input_features"] = tf.zeros((32, 100, 13))
x["input_points"] = tf.zeros((32, 100, 2))
x["input_mask"] = tf.zeros((32, 100))
x["input_jet"] = tf.zeros((32, 4))
x["input_time"] = tf.zeros((32, 1))

model(x)
out = model.body(x)
print(len(out), out.shape)

Load the weights of the backbone model.

In [ ]:
model.summary()
model.load_weights("checkpoints/PET_jetclass_8_local_layer_scale_token_baseline_all.weights.h5", by_name=True)

In [ ]:
data = awkward.from_parquet("zh.parquet")

In [ ]:
reco_cand_p4s = to_p4(data["reco_cand_p4s"])
reco_jet_p4s = to_p4(data["reco_jet_p4s"])

delta_eta = reco_cand_p4s.eta - reco_jet_p4s.eta
delta_phi = deltaphi(reco_cand_p4s.phi, reco_jet_p4s.phi)
log_pt = np.log(reco_cand_p4s.pt)
log_e = np.log(reco_cand_p4s.energy)
log_ptjet = np.log(1 - reco_cand_p4s.pt/reco_jet_p4s.pt)
log_ejet = np.log(1 - reco_cand_p4s.energy/reco_jet_p4s.energy)
delta_r = deltar(reco_cand_p4s.eta, reco_cand_p4s.phi, reco_jet_p4s.eta, reco_jet_p4s.phi)
charge = data["reco_cand_charge"]
is_ele = np.abs(data["reco_cand_pdg"])==11
is_mu = np.abs(data["reco_cand_pdg"])==13
is_photon = np.abs(data["reco_cand_pdg"])==22
is_chhad = np.abs(data["reco_cand_pdg"])==210
is_nhad = np.abs(data["reco_cand_pdg"])==130

In [ ]:
pad_size = 64 #max number of particles per jet
fill_val = 0 #fill value of padded data

#create particle array in the shape [njets, pad_size, 13]
vals = [
    awkward.to_numpy(
        awkward.fill_none(
            awkward.pad_none(
                x, pad_size, clip=True), fill_val
        )
    ) for x in [delta_eta, delta_phi, log_pt, log_e, log_ptjet, log_ejet, delta_r, charge, is_ele, is_mu, is_photon, is_chhad, is_nhad]
]
particles = np.stack(vals, axis=-1)
particles[np.isnan(particles)] = 0
particles[np.isinf(particles)] = 0
particles_mask = (~awkward.to_numpy(awkward.pad_none(delta_eta, pad_size, clip=True)).mask).astype(np.float32)

#normalize particles
means_particle = particles[np.squeeze(particles_mask==1)].mean(axis=0)
stds_particle = particles[np.squeeze(particles_mask==1)].std(axis=0)
stds_particle[stds_particle==0] = 1
particles = (particles - means_particle)/stds_particle

#create jet array in the shape [njets, 4]
jets = awkward.to_numpy(np.stack([
    reco_jet_p4s.pt,
    reco_jet_p4s.eta,
    reco_jet_p4s.mass,
    awkward.num(reco_cand_p4s)], axis=-1)
)
jets[np.isnan(jets)] = 0
jets[np.isinf(jets)] = 0

#normalize jets
means_jet = jets.mean(axis=0)
stds_jet = jets.std(axis=0)
stds_jet[stds_jet==0] = 1
jets = (jets - means_jet)/stds_jet


In [ ]:
means_particle, stds_particle

In [ ]:
means_jet, stds_jet

In [ ]:
particles.shape, particles_mask.shape, jets.shape

In [ ]:
targets = awkward.to_numpy(data["gen_jet_tau_decaymode"])

In [ ]:
def prepare_data(start, stop):
    x = {}
    x["input_features"] = particles[start:stop]
    x["input_points"] = particles[start:stop, :, :2]
    x["input_mask"] = np.expand_dims(particles_mask[start:stop], axis=-1)
    x["input_jet"] = jets[start:stop]
    x["input_time"] = np.zeros((stop-start, 1))
    y = targets[start:stop]
    return x, y

In [ ]:
from tensorflow.keras import layers
from layers import StochasticDepth, TalkingHeadAttention, LayerScale, RandomDrop

def get_encoding(x, projection_dim, use_bias=True):
    x = layers.Dense(2*projection_dim, use_bias=use_bias, activation='gelu')(x)
    x = layers.Dense(projection_dim, use_bias=use_bias, activation='gelu')(x)
    return x

def FourierProjection(x,projection_dim,num_embed=64):    
    half_dim = num_embed // 2
    emb = tf.math.log(10000.0) / (half_dim - 1)
    emb = tf.cast(emb,tf.float32)
    freq = tf.exp(-emb* tf.range(start=0, limit=half_dim, dtype=tf.float32))


    angle = x*freq*1000.0
    embedding = tf.concat([tf.math.sin(angle),tf.math.cos(angle)],-1)*x
    embedding = layers.Dense(2*projection_dim,activation="swish",use_bias=False)(embedding)
    embedding = layers.Dense(projection_dim,activation="swish",use_bias=False)(embedding)
    
    return embedding

def knn(num_points, k, topk_indices, features):
    # topk_indices: (N, P, K)
    # features: (N, P, C)    
    batch_size = tf.shape(features)[0]

    batch_indices = tf.reshape(tf.range(batch_size), (-1, 1, 1))
    batch_indices = tf.tile(batch_indices, (1, num_points, k))
    indices = tf.stack([batch_indices, topk_indices], axis=-1)
    return tf.gather_nd(features, indices)

def get_neighbors(points,features,projection_dim,K):
    drij = pairwise_distance(points)  # (N, P, P)
    _, indices = tf.nn.top_k(-drij, k=K + 1)  # (N, P, K+1)
    indices = indices[:, :, 1:]  # (N, P, K)
    knn_fts = knn(tf.shape(points)[1], K, indices, features)  # (N, P, K, C)
    knn_fts_center = tf.broadcast_to(tf.expand_dims(features, 2), tf.shape(knn_fts))
    local = tf.concat([knn_fts-knn_fts_center,knn_fts_center],-1)
    local = layers.Dense(2*projection_dim,activation='gelu')(local)
    local = layers.Dense(projection_dim,activation='gelu')(local)
    local = tf.reduce_mean(local,-2)
    
    return local

def pairwise_distance(point_cloud):
    r = tf.reduce_sum(point_cloud * point_cloud, axis=2, keepdims=True)
    m = tf.matmul(point_cloud, point_cloud, transpose_b = True)
    D = r - 2 * m + tf.transpose(r, perm=(0, 2, 1)) + 1e-5
    return D


class TransformerModel(keras.Model):
    def __init__(self,
                 use_backbone,
                 num_feat,
                 num_jet,      
                 num_classes=2):
        
        super(TransformerModel, self).__init__()
        
        self.projection_dim = model.projection_dim
        self.num_heads = model.num_heads
        
        self.feature_drop = model.feature_drop
        self.num_keep = model.num_keep
        self.mode = model.mode
        self.num_layers = model.num_layers
        self.layer_scale = model.layer_scale
        self.layer_scale_init = model.layer_scale_init
        self.drop_probability = model.drop_probability
        self.dropout = model.dropout
        
        self._input_features = layers.Input(shape=(None, num_feat), name='input_features')
        self._input_points = layers.Input(shape=(None, 2), name='input_points')
        self._input_mask = layers.Input(shape=(None, 1), name='input_mask')
        self._input_jet = layers.Input((num_jet, ),name='input_jet')
        self._input_time = layers.Input((None, ),name='input_time')

        if use_backbone:
            self.backbone_body = self.PET_body(
                self._input_features,
                self._input_points,
                self._input_mask,
                self._input_time,
                True,
                10,
                2,
                False
            )
            self.backbone = keras.Model(
                inputs=[self._input_features, self._input_points, self._input_mask, self._input_time],
                outputs=[self.backbone_body], name="backbone"
            )

            particles_encoded = self.backbone_body
        else:
            particles_encoded = get_encoding(self._input_features, self.projection_dim)

        for ilayer in range(2):
            updates = layers.MultiHeadAttention(
                num_heads=self.num_heads,
                key_dim=self.projection_dim//self.num_heads)(
                query=particles_encoded, value=particles_encoded, key=particles_encoded)
            particles_encoded = layers.Add()([updates, particles_encoded])
            particles_encoded = layers.GroupNormalization(groups=1)(particles_encoded)
            particles_encoded = layers.Dense(self.projection_dim)(particles_encoded)
        
        representation = layers.GlobalAveragePooling1D()(particles_encoded)
        jet_encoded = get_encoding(self._input_jet, self.projection_dim)
        representation = layers.Dense(self.projection_dim,activation='gelu')(representation+jet_encoded)
        outputs_dm_pred = layers.Dense(num_classes, activation="softmax")(representation)
        
        self.decaymode_head = keras.Model(
            inputs=[self._input_features, self._input_points, self._input_mask, self._input_jet, self._input_time],
            outputs=[outputs_dm_pred], name="decaymode_head"
        )

    def PET_body(self,
                 input_features,
                 input_points,
                 input_mask,
                 input_time,
                 local, K,num_local,
                 talking_head,
                 ):
            
        #Randomly drop features not present in other datasets
        encoded = RandomDrop(self.feature_drop if  'all' in self.mode else 0.0,num_skip=self.num_keep)(input_features)                        
        encoded = get_encoding(encoded,self.projection_dim)

        time = FourierProjection(input_time,self.projection_dim)
        time = tf.tile(time[:,None, :], [1,tf.shape(encoded)[1], 1])*input_mask
        time = layers.Dense(2*self.projection_dim,activation='gelu',use_bias=False)(time)
        scale,shift = tf.split(time,2,-1)
        
        encoded = encoded*(1.0+scale) + shift
        
        if local:
            coord_shift = tf.multiply(999., tf.cast(tf.equal(input_mask, 0), dtype='float32'))        
            points = input_points[:,:,:2]
            local_features = input_features
            for _ in range(num_local):
                local_features = get_neighbors(coord_shift+points,local_features,self.projection_dim,K)
                points = local_features
                
            encoded = layers.Add()([local_features,encoded])

        skip_connection = encoded
        for i in range(self.num_layers):
            x1 = layers.GroupNormalization(groups=1)(encoded)
            if talking_head:
                updates, _ = TalkingHeadAttention(self.projection_dim, self.num_heads, 0.0)(x1)
            else:
                updates = layers.MultiHeadAttention(num_heads=self.num_heads,
                                                    key_dim=self.projection_dim//self.num_heads)(x1,x1)

            if self.layer_scale:
                updates = LayerScale(self.layer_scale_init, self.projection_dim)(updates,input_mask)
            updates = StochasticDepth(self.drop_probability)(updates)
            x2 = layers.Add()([updates,encoded])
            x3 = layers.GroupNormalization(groups=1)(x2)
            x3 = layers.Dense(2*self.projection_dim,activation="gelu")(x3)
            x3 = layers.Dropout(self.dropout)(x3)
            x3 = layers.Dense(self.projection_dim)(x3)
            if self.layer_scale:
                x3 = LayerScale(self.layer_scale_init, self.projection_dim)(x3,input_mask)
            x3 = StochasticDepth(self.drop_probability)(x3)
            encoded = layers.Add()([x3,x2])*input_mask
        return encoded + skip_connection
    
    def call(self, x):
        ret = self.decaymode_head([
            x["input_features"], x["input_points"], x["input_mask"], x["input_jet"], x["input_time"]
        ])
        return ret

In [ ]:
model_dm_direct = TransformerModel(False, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_direct(x)

model_dm_bb = TransformerModel(True, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_bb(x)

#Load the weights of the backbone, freeze
model_dm_bb.backbone.set_weights(model.body.weights)
model_dm_bb.backbone.trainable = False

In [ ]:
model_dm_direct.summary()

In [ ]:
X_train, y_train = prepare_data(0, 400000)
X_val, y_val = prepare_data(400000, 500000)

In [ ]:
model_dm_direct.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history = model_dm_direct.fit(
    X_train, y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
model_dm_bb = TransformerModel(True, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_bb(x)
#model_dm_bb.backbone.set_weights(model.body.weights)
model_dm_bb.backbone.trainable = True
model_dm_bb.summary()

In [ ]:
model_dm_bb.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history2 = model_dm_bb.fit(
    X_train, y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

In [ ]:
plt.plot(history2.history["loss"])
plt.plot(history2.history["val_loss"])

In [ ]:
model_dm_bb_cp = TransformerModel(True, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_bb_cp(x)
model_dm_bb_cp.backbone.set_weights(model.body.weights)
model_dm_bb_cp.backbone.trainable = True
model_dm_bb_cp.summary()

In [ ]:
model_dm_bb_cp.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history3 = model_dm_bb_cp.fit(
    X_train, y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

In [ ]:
plt.plot(history3.history["loss"])
plt.plot(history3.history["val_loss"])

In [ ]:
plt.plot(history.history["loss"], label="direct", color="black")
plt.plot(history2.history["loss"], label="OmniLearn naive", color="red")
plt.plot(history3.history["loss"], label="OmniLearn checkpoint", color="orange")
plt.legend()
plt.ylabel("train loss")

In [ ]:
plt.plot(history.history["val_loss"], label="direct", color="black")
plt.plot(history2.history["val_loss"], label="OmniLearn naive", color="red")
plt.plot(history3.history["val_loss"], label="OmniLearn checkpoint", color="orange")
plt.legend()
plt.ylabel("validation loss")